In [1]:
!wget "https://pancanatlas.xenahubs.net/download/TCGA_phenotype_denseDataOnlyDownload.tsv.gz"

--2022-12-30 16:17:52--  https://pancanatlas.xenahubs.net/download/TCGA_phenotype_denseDataOnlyDownload.tsv.gz
Resolving pancanatlas.xenahubs.net (pancanatlas.xenahubs.net)... 18.214.79.84, 34.205.57.191, 52.21.210.223, ...
Connecting to pancanatlas.xenahubs.net (pancanatlas.xenahubs.net)|18.214.79.84|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://tcga-pancan-atlas-hub.s3.us-east-1.amazonaws.com:443/download/TCGA_phenotype_denseDataOnlyDownload.tsv.gz [following]
--2022-12-30 16:17:54--  https://tcga-pancan-atlas-hub.s3.us-east-1.amazonaws.com/download/TCGA_phenotype_denseDataOnlyDownload.tsv.gz
Resolving tcga-pancan-atlas-hub.s3.us-east-1.amazonaws.com (tcga-pancan-atlas-hub.s3.us-east-1.amazonaws.com)... 52.217.130.138, 54.231.232.106, 52.217.38.56, ...
Connecting to tcga-pancan-atlas-hub.s3.us-east-1.amazonaws.com (tcga-pancan-atlas-hub.s3.us-east-1.amazonaws.com)|52.217.130.138|:443... connected.
HTTP request sent, awaiting respon

In [2]:
!gunzip TCGA_phenotype_denseDataOnlyDownload.tsv.gz

In [3]:
!wget "https://legacy.xenahubs.net/download/TCGA.PANCAN.sampleMap/HiSeqV2.gz"

--2022-12-30 16:18:23--  https://legacy.xenahubs.net/download/TCGA.PANCAN.sampleMap/HiSeqV2.gz
Resolving legacy.xenahubs.net (legacy.xenahubs.net)... 54.83.19.224
Connecting to legacy.xenahubs.net (legacy.xenahubs.net)|54.83.19.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 513041354 (489M) [application/gzip]
Saving to: ‘HiSeqV2.gz’

HiSeqV2.gz          100%[===================>] 489.27M  3.89MB/s    in 2m 5s   

2022-12-30 16:20:30 (3.91 MB/s) - ‘HiSeqV2.gz’ saved [513041354/513041354]



In [4]:
!gunzip 'HiSeqV2.gz'

In [2]:
import pandas as pd
import h5py
import numpy as np
import progressbar

In [2]:
data = 'HiSeqV2'
labels = 'TCGA_phenotype_denseDataOnlyDownload.tsv'
dbPath = 'data.h5'
verbose = False

In [3]:
print('Loading data ... Patience.')
df = pd.read_csv(data, sep='\t').transpose()

print('Loading labels ...')
labeldf = pd.read_csv(labels, sep = '\t')

print('Housekeeping ...')
df.columns = df.iloc[0]
df = df.drop('Sample', axis = 0)

labeldf = labeldf.set_index('sample')

# dimensions: 10459 x 20530

nTotal = df.shape[0]    #10459
nFeat = df.shape[1]     #20530

Loading data ... Patience.
Loading labels ...
Housekeeping ...


In [4]:
nFeat

20530

In [5]:
df

Sample,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,REM1,MTVR2,RTN4RL2,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
TCGA-S9-A7J2-01,10.9576,4.8099,0.4657,11.2675,10.1761,10.4769,13.0456,3.2299,0.4657,8.7533,...,0.0,1.3357,2.9741,0.0,9.2594,9.4779,6.1595,9.6465,0.0,9.4848
TCGA-G3-A3CH-11,11.0186,5.3847,0.0,11.669,11.398,10.8249,11.5487,3.5408,1.4714,7.9144,...,0.0,3.5408,5.5302,0.0,7.5066,10.5302,7.3741,13.0045,7.0466,10.3411
TCGA-EK-A2RE-01,9.7106,2.8888,0.4192,11.4903,11.7371,9.9473,10.841,2.5988,0.0,3.9541,...,0.7436,0.0,2.953,1.2319,9.5217,13.8492,6.5812,9.2958,0.4192,9.745
TCGA-44-6778-01,9.6205,7.9642,1.5378,11.8432,11.0531,10.9005,12.4145,4.5366,2.0609,4.1805,...,0.0,2.0609,6.1839,4.1291,8.9832,12.3412,9.0862,10.4779,9.4517,10.4395
TCGA-VM-A8C8-01,11.6596,8.5622,0.0,11.2677,11.3549,10.8579,13.256,5.9962,0.0,5.357,...,1.3549,0.0,4.753,0.6034,9.0573,8.8984,5.9116,9.9584,1.6216,9.6811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-95-7947-01,10.0459,6.6572,0.0,11.3827,10.9459,10.5165,12.5061,3.6214,0.6881,3.1611,...,3.758,0.9387,4.2502,0.0,9.566,10.8301,7.6826,11.3541,7.3565,10.3328
TCGA-VQ-AA6F-01,9.5758,5.8461,0.0,11.6292,10.6314,11.5036,12.5995,5.9237,0.9655,5.556,...,0.3979,0.3979,4.74,5.4128,8.9264,11.2215,7.8272,10.8221,7.0938,10.614
TCGA-55-6985-11,9.6575,8.9521,0.4791,11.6766,11.3748,11.3349,12.3318,5.6618,0.4791,4.7755,...,0.8381,2.5176,6.8133,1.1254,9.0194,13.5597,9.9022,10.9969,9.3046,10.2187
TCGA-DD-A115-01,11.7589,3.7591,0.0,12.0914,11.5774,10.1702,12.0789,4.3081,0.0,11.0616,...,2.8619,2.8619,4.1258,0.0,7.5925,11.2591,8.0674,13.3772,6.4848,9.8594


In [6]:
df.to_csv('parsed_tcga_data.csv') # data is ready to be used in the next notebook

In [13]:
print('Total Number of samples: '+ str(nTotal))
print('Features (RNASeq) per sample: ' + str(nFeat))

print('Diseases to predict: ')

diseases = labeldf._primary_disease.unique()

for disease in diseases:
    print(disease)

Total Number of samples: 10459
Features (RNASeq) per sample: 20530
Diseases to predict: 
skin cutaneous melanoma
thyroid carcinoma
sarcoma
prostate adenocarcinoma
pheochromocytoma & paraganglioma
pancreatic adenocarcinoma
head & neck squamous cell carcinoma
esophageal carcinoma
colon adenocarcinoma
cervical & endocervical cancer
breast invasive carcinoma
bladder urothelial carcinoma
testicular germ cell tumor
kidney papillary cell carcinoma
kidney clear cell carcinoma
acute myeloid leukemia
rectum adenocarcinoma
ovarian serous cystadenocarcinoma
lung adenocarcinoma
liver hepatocellular carcinoma
uterine corpus endometrioid carcinoma
glioblastoma multiforme
brain lower grade glioma
uterine carcinosarcoma
thymoma
stomach adenocarcinoma
diffuse large B-cell lymphoma
lung squamous cell carcinoma
mesothelioma
kidney chromophobe
uveal melanoma
cholangiocarcinoma
adrenocortical cancer
